In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score
from sklearn.model_selection import train_test_split,GridSearchCV




In [3]:
df=pd.read_csv("C:\\Users\\ritik\\Downloads\\Loan_Data.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
df.shape

(614, 13)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [ ]:
import missingno as msno
msno.bar(df)

In [6]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [10]:
for i in df.columns:
    if df[i].isnull().sum()>0:
        if df[i].dtype=="object":
            df[i]=df[i].fillna(df[i].mode()[0])
        elif ((df[i].dtype=='int')  or (df[i].dtype=='float')):
            df[i]=df[i].fillna(df[i].mean())

            
        

In [11]:
df.isnull().sum().sum()
            

0

In [15]:
df.select_dtypes(include="object").columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Property_Area', 'Loan_Status'],
      dtype='object')

In [16]:
df.drop(columns="Loan_ID",inplace=True)

In [17]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y


In [18]:
columns=df.select_dtypes(include="object").columns
columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'Property_Area', 'Loan_Status'],
      dtype='object')

In [19]:
for i in columns:
    print(i,"----------",df[i].unique())

Gender ---------- ['Male' 'Female']
Married ---------- ['No' 'Yes']
Dependents ---------- ['0' '1' '2' '3+']
Education ---------- ['Graduate' 'Not Graduate']
Self_Employed ---------- ['No' 'Yes']
Property_Area ---------- ['Urban' 'Rural' 'Semiurban']
Loan_Status ---------- ['Y' 'N']


In [20]:
df["Dependents"]=df["Dependents"].replace({"0":0,"1":1,"2":2,"3+":3})
df["Dependents"].unique()

array([0, 1, 2, 3], dtype=int64)

In [21]:
## Seggregate x and y

In [22]:
x=df.drop(columns="Loan_Status")
y=df["Loan_Status"]

In [23]:
x.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban


In [24]:
l=x.select_dtypes(include="object").columns
l

Index(['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area'], dtype='object')

In [25]:
x=pd.get_dummies(columns=l,data=x,drop_first=True)

In [26]:
x

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,0,5849,0.0,146.412162,360.0,1.0,1,0,0,0,0,1
1,1,4583,1508.0,128.000000,360.0,1.0,1,1,0,0,0,0
2,0,3000,0.0,66.000000,360.0,1.0,1,1,0,1,0,1
3,0,2583,2358.0,120.000000,360.0,1.0,1,1,1,0,0,1
4,0,6000,0.0,141.000000,360.0,1.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,2900,0.0,71.000000,360.0,1.0,0,0,0,0,0,0
610,3,4106,0.0,40.000000,180.0,1.0,1,1,0,0,0,0
611,1,8072,240.0,253.000000,360.0,1.0,1,1,0,0,0,1
612,2,7583,0.0,187.000000,360.0,1.0,1,1,0,0,0,1


In [27]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1200)

In [43]:
abc=AdaBoostClassifier()
abc.fit(x_train,y_train)

AdaBoostClassifier()

In [44]:
y_pred=abc.predict(x_test)
accuracy_score(y_test,y_pred)

0.7792207792207793

In [45]:
abd=AdaBoostClassifier(base_estimator=LogisticRegression())
abd.fit(x_train,y_train)

AdaBoostClassifier(base_estimator=LogisticRegression())

In [46]:
y_pred=abc.predict(x_test)
accuracy_score(y_test,y_pred)

0.7792207792207793

- they are high bias and low variance 
- because the are based on assumption

In [47]:
 grid_param = {
     'n_estimators' :[10,50,100,200,500],
     'learning_rate' :[0.0001,0.01,0.001,1.0]
}


In [51]:
grid_search = GridSearchCV(estimator=abc,
                     param_grid=grid_param,
                    n_jobs =-1,cv=10,scoring="accuracy") 

In [52]:
grid_search.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=AdaBoostClassifier(), n_jobs=-1,
             param_grid={'learning_rate': [0.0001, 0.01, 0.001, 1.0],
                         'n_estimators': [10, 50, 100, 200, 500]},
             scoring='accuracy')

In [54]:
best_parameters = grid_search.best_params_
print(best_parameters)

{'learning_rate': 0.0001, 'n_estimators': 10}


In [57]:
treemodel=AdaBoostClassifier(**best_parameters)
treemodel.fit(x_train,y_train)

AdaBoostClassifier(learning_rate=0.0001, n_estimators=10)

In [58]:
y_pred=treemodel.predict(x_test)
accuracy_score(y_test,y_pred)

0.7987012987012987